## Installere pakker


In [36]:
#pip install -r requirements.txt



## Importering av pakker

In [123]:
import pyarrow as pa
import pyarrow.parquet as pq
from IPython.display import display
import folium
import time
import pandas as pd
import duckdb
import os
import duckdb
import folium
import pandas as pd
from shapely import wkt
from IPython.display import display
from folium.plugins import MarkerCluster
import os
import pandas as pd
import geopandas as gpd
from shapely import wkb
import time
from datetime import datetime
import duckdb
from datetime import datetime


# Partisjonering

## Partisjonering etter ship type

In [38]:


# Opprett DuckDB-tilkobling
con = duckdb.connect()

# Les inn data fra en Parquet-fil
query = """
    SELECT *
    FROM read_parquet('/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/hais_2024-12-01.snappy.parquet.geo.parquet')
"""
data = con.execute(query).fetchdf()

# Lukk tilkoblingen
con.close()

# Konverter Pandas DataFrame til PyArrow Table
table = pa.Table.from_pandas(data)

# Definer output path
output_path = '/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataShipType'

# Skriv til partisjonerte Parquet-filer
pq.write_to_dataset(
    table,
    root_path=output_path,
    partition_cols=['ship_type']  # Partisjonering
)

print(f"Data er skrevet til {output_path} i partisjonerte Parquet-filer.")


Data er skrevet til /Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataShipType i partisjonerte Parquet-filer.


## Partisjonering etter timer

In [49]:
def partition_parquet(file_path, output_dir):
    # Koble til DuckDB
    conn = duckdb.connect()

    # Opprett katalogen hvis den ikke eksisterer
    os.makedirs(output_dir, exist_ok=True)

    # SQL-spørring for å hente dataene og partisjonere etter timer
    query = f"""
    SELECT *, EXTRACT(HOUR FROM date_time_utc) AS hour
    FROM read_parquet('{file_path}')
    """

    # Kjør spørringen og hent resultatene
    df = conn.execute(query).fetchdf()

    # Sjekk om vi fikk data
    if df.empty:
        print("Ingen data funnet i filen. Sjekk at filen inneholder gyldige rader.")
        conn.close()
        return False

    # Lagre partisjonerte filer
    saved_files = []
    for hour in df['hour'].unique():
        partition_df = df[df['hour'] == hour]
        output_file = os.path.join(output_dir, f'partitioned_{hour}.parquet')

        partition_df.to_parquet(output_file)
        saved_files.append(output_file)

    # Lukk tilkoblingen
    conn.close()

    # Skriv ut status
    if saved_files:
        print(f"Partisjonering fullført! Filene ble lagret i: {output_dir}")
        print(f"Lagrede filer: {saved_files}")
        return True
    else:
        print("Ingen filer ble lagret. Sjekk inputdata.")
        return False

# Eksempel på bruk
file_path = '/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/hais_2024-12-01.snappy.parquet.geo.parquet'
output_dir = '/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataTimer'

partition_parquet(file_path, output_dir)


✅ Partisjonering fullført! Filene ble lagret i: /Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataTimer2
Lagrede filer: ['/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataTimer2/partitioned_5.parquet', '/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataTimer2/partitioned_4.parquet', '/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataTimer2/partitioned_3.parquet', '/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataTimer2/partitioned_2.parquet', '/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataTimer2/partitioned_1.parquet', '/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataTimer2/partitioned_0.parquet', '/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataTimer2/partitioned_11.parquet', '/Users/johannehaakenstad/Bachelor-Filer

True

# Filtrering

##  Setter en limit til kun 5 rader, den henter dermet kun de første 5 radene i filen og setter det i et kart


In [41]:

# Opprett DuckDB-tilkobling
con = duckdb.connect()

# Leser fra parquet-filen
query = """
    SELECT *
    FROM read_parquet('/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/hais_2024-12-01.snappy.parquet.geo.parquet')
"""

# Definer batch-størrelse til 100 rader
batch_size = 5
offset = 0

# Bygg spørring for å hente 5 rader
batch_query = f"{query} LIMIT {batch_size} OFFSET {offset}"
batch = con.execute(batch_query).fetchdf()

# Sjekk om vi har fått noen data
if len(batch) == 0:
    print("Ingen data funnet.")
else:
    print("Streamer 5 rader:")
    display(batch.head())  # Viser de første radene i en tabell

    # Hvis filen inneholder geodata, for eksempel med 'latitude' og 'longitude'-kolonner,
    # kan vi lage et enkelt Folium-kart:
    if "latitude" in batch.columns and "longitude" in batch.columns:
        avg_lat = batch["latitude"].mean()
        avg_lon = batch["longitude"].mean()
        m = folium.Map(location=[avg_lat, avg_lon], zoom_start=7)

        for _, row in batch.iterrows():
            lat = row["latitude"]
            lon = row["longitude"]
            if not pd.isna(lat) and not pd.isna(lon):
                folium.Marker(
                    location=[lat, lon],
                    popup=str(row.iloc[:5].to_dict())
                ).add_to(m)

        display(m)
    else:
        print("Ingen 'latitude'/'longitude'-kolonner funnet.")

# Lukk tilkoblingen
con.close()



Streamer 5 rader:


,date_time_utc,mmsi,longitude,latitude,status,course_over_ground,speed_over_ground,rate_of_turn,maneuvre,imo,callsign,ship_name,ship_type,length,draught,data_source,ais_class,hex_7,hex_14,geometry
0,2024-12-01 05:58:44,257565700,7.476903,58.020340,5,215.0,0.0,-9.0,0.0,0,LF5484,RESCUE 157 BILL,51,14,0.0,G,A,608154275335372799,639679472713910703,"[1, 1, 0, 0, 0, 97, 208, 182, 88, 89, 232, 29,..."
1,2024-12-01 05:30:48,257565700,7.476915,58.020353,5,3.2,0.0,16.0,0.0,0,LF5484,RESCUE 157 BILL,51,14,0.0,G,A,608154275335372799,639679472713910327,"[1, 1, 0, 0, 0, 42, 227, 223, 103, 92, 232, 29..."
2,2024-12-01 05:16:51,257565700,7.476912,58.020327,5,55.2,0.1,15.0,0.0,0,LF5484,RESCUE 157 BILL,51,14,0.0,G,A,608154275335372799,639679472713910663,"[1, 1, 0, 0, 0, 209, 31, 187, 135, 91, 232, 29..."
3,2024-12-01 05:12:51,257565700,7.476917,58.020340,5,134.2,0.0,16.0,0.0,0,LF5484,RESCUE 157 BILL,51,14,0.0,G,A,608154275335372799,639679472713910671,"[1, 1, 0, 0, 0, 95, 120, 70, 215, 92, 232, 29,..."
4,2024-12-01 05:06:46,257565700,7.476915,58.020327,5,26.6,0.0,10.0,0.0,0,LF5484,RESCUE 157 BILL,51,14,0.0,G,A,608154275335372799,639679472713910663,"[1, 1, 0, 0, 0, 42, 227, 223, 103, 92, 232, 29..."


# Filtrering av kolonner


In [42]:
# Opprett DuckDB-tilkobling
con = duckdb.connect()

# Velg spesifikke kolonner
query = """
    SELECT date_time_utc, geometry, ship_name
    FROM read_parquet('/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/hais_2024-12-01.snappy.parquet.geo.parquet')
"""

# Utfør spørringen
filtered_data = con.execute(query).fetchdf()

# Vis de første radene
display(filtered_data.head())

# Lukk tilkoblingen
con.close()


,date_time_utc,geometry,ship_name
0,2024-12-01 05:58:44,"[1, 1, 0, 0, 0, 97, 208, 182, 88, 89, 232, 29,...",RESCUE 157 BILL
1,2024-12-01 05:30:48,"[1, 1, 0, 0, 0, 42, 227, 223, 103, 92, 232, 29...",RESCUE 157 BILL
2,2024-12-01 05:16:51,"[1, 1, 0, 0, 0, 209, 31, 187, 135, 91, 232, 29...",RESCUE 157 BILL
3,2024-12-01 05:12:51,"[1, 1, 0, 0, 0, 95, 120, 70, 215, 92, 232, 29,...",RESCUE 157 BILL
4,2024-12-01 05:06:46,"[1, 1, 0, 0, 0, 42, 227, 223, 103, 92, 232, 29...",RESCUE 157 BILL


## Filtrering av tidsstempler (og visualisering i kart)

In [43]:


# Opprett DuckDB-tilkobling
con = duckdb.connect()

# Definer tidsintervallet
start_time = "2024-12-01 05:12:51"
end_time = "2024-12-01 05:14:00"

# SQL-spørring for å filtrere data basert på tidspunkt
query = f"""
    SELECT *
    FROM read_parquet('/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/hais_2024-12-01.snappy.parquet.geo.parquet')
    WHERE CAST(date_time_utc AS TIMESTAMP)
    BETWEEN TIMESTAMP '{start_time}' AND TIMESTAMP '{end_time}'
"""

# Utfør spørringen og hent dataen
filtered_data = con.execute(query).fetchdf()

# Lukk tilkoblingen
con.close()


# Sjekk om vi fikk noen rader
if filtered_data.empty:
    print("Ingen data funnet i det spesifiserte tidsintervallet.")
else:
    print(f"Fant {len(filtered_data)} rader i tidsintervallet:")
    display(filtered_data)  # Viser hele den filtrerte tabellen

    # Sjekk om datasettet inneholder 'latitude' og 'longitude'
    if "latitude" in filtered_data.columns and "longitude" in filtered_data.columns:
        avg_lat = filtered_data["latitude"].mean()
        avg_lon = filtered_data["longitude"].mean()

        # Lag et Folium-kart med gjennomsnittlig senterpunkt
        m = folium.Map(location=[avg_lat, avg_lon], zoom_start=10)

        # Legg til punktene i kartet
        for _, row in filtered_data.iterrows():
            lat, lon = row["latitude"], row["longitude"]
            if not pd.isna(lat) and not pd.isna(lon):  # Sjekk at verdiene ikke er NaN
                ship_name = row.get("ship_name", "Ukjent skip")  # Erstatt med riktig kolonnenavn for skipets navn
                popup_text = f"Timestamp: {row['date_time_utc']}<br>Ship: {ship_name}"

                folium.Marker(
                    location=[lat, lon],
                    popup=popup_text,
                ).add_to(m)

        # Vis kartet
        display(m)
    else:
        print("Ingen 'latitude'/'longitude'-kolonner funnet i datasettet.")


Fant 15 rader i tidsintervallet:


,date_time_utc,mmsi,longitude,latitude,status,course_over_ground,speed_over_ground,rate_of_turn,maneuvre,imo,callsign,ship_name,ship_type,length,draught,data_source,ais_class,hex_7,hex_14,geometry
0,2024-12-01 05:12:51,257565700,7.476917,58.020340,5,134.2,0.0,16.0,0.0,0,LF5484,RESCUE 157 BILL,51,14,0.0,G,A,608154275335372799,639679472713910671,"[1, 1, 0, 0, 0, 95, 120, 70, 215, 92, 232, 29,..."
1,2024-12-01 05:13:58,258012180,8.034652,58.146895,15,360.0,0.0,-128.0,0.0,0,LF7245,SAR NICOLAI JARLSBY,51,9,0.6,G,A,608155171775578111,639680369155664551,"[1, 1, 0, 0, 0, 123, 231, 196, 220, 189, 17, 3..."
2,2024-12-01 05:13:47,258012180,8.034647,58.146887,15,360.0,0.0,-128.0,0.0,0,LF7245,SAR NICOLAI JARLSBY,51,9,0.6,G,A,608155171775578111,639680369155664783,"[1, 1, 0, 0, 0, 52, 59, 255, 52, 189, 17, 32, ..."
3,2024-12-01 05:13:38,258012180,8.034633,58.146860,15,360.0,0.0,-128.0,0.0,0,LF7245,SAR NICOLAI JARLSBY,51,9,0.6,G,A,608155171775578111,639680369155664791,"[1, 1, 0, 0, 0, 53, 103, 183, 117, 187, 17, 32..."
4,2024-12-01 05:13:28,258012180,8.034620,58.146840,15,360.0,0.0,-128.0,0.0,0,LF7245,SAR NICOLAI JARLSBY,51,9,0.6,G,A,608155171775578111,639680369155662183,"[1, 1, 0, 0, 0, 54, 147, 111, 182, 185, 17, 32..."
5,2024-12-01 05:13:18,258012180,8.034605,58.146835,15,360.0,0.0,-128.0,0.0,0,LF7245,SAR NICOLAI JARLSBY,51,9,0.6,G,A,608155171775578111,639680369155662183,"[1, 1, 0, 0, 0, 97, 142, 30, 191, 183, 17, 32,..."
6,2024-12-01 05:13:08,258012180,8.034602,58.146838,15,360.0,0.0,-128.0,0.0,0,LF7245,SAR NICOLAI JARLSBY,51,9,0.6,G,A,608155171775578111,639680369155662183,"[1, 1, 0, 0, 0, 180, 44, 12, 79, 183, 17, 32, ..."
7,2024-12-01 05:12:58,258012180,8.034608,58.146845,15,360.0,0.0,-128.0,0.0,0,LF7245,SAR NICOLAI JARLSBY,51,9,0.6,G,A,608155171775578111,639680369155662183,"[1, 1, 0, 0, 0, 209, 9, 219, 46, 184, 17, 32, ..."
8,2024-12-01 05:14:00,258258500,8.778695,58.461892,0,221.7,0.0,0.0,0.0,0,LDLN,RESCUE STORMBULL,51,17,1.0,G,A,608155183335079935,639680380713776863,"[1, 1, 0, 0, 0, 17, 30, 109, 28, 177, 142, 33,..."
9,2024-12-01 05:13:50,258258500,8.778688,58.461893,0,221.4,0.0,0.0,0.0,0,LDLN,RESCUE STORMBULL,51,17,1.0,G,A,608155183335079935,639680380713776863,"[1, 1, 0, 0, 0, 243, 64, 158, 60, 176, 142, 33..."


## Filtrering av de partisjonerte filene etter tid

In [44]:
# Opprett DuckDB-tilkobling
con = duckdb.connect()

# Definer tidsintervallet
start_time = "2024-12-01 11:12:51"
end_time = "2024-12-01 11:14:00"

# SQL-spørring for å filtrere data basert på tidspunkt
query = f"""
    SELECT *
    FROM read_parquet('/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataTimer/partitioned_11.parquet')
    WHERE CAST(date_time_utc AS TIMESTAMP)
    BETWEEN TIMESTAMP '{start_time}' AND TIMESTAMP '{end_time}'
"""

# Utfør spørringen og hent dataen
filtered_data = con.execute(query).fetchdf()

# Lukk tilkoblingen
con.close()


# Sjekk om vi fikk noen rader
if filtered_data.empty:
    print("Ingen data funnet i det spesifiserte tidsintervallet.")
else:
    print(f"Fant {len(filtered_data)} rader i tidsintervallet:")
    display(filtered_data)  # Viser hele den filtrerte tabellen

    # Sjekk om datasettet inneholder 'latitude' og 'longitude'
    if "latitude" in filtered_data.columns and "longitude" in filtered_data.columns:
        avg_lat = filtered_data["latitude"].mean()
        avg_lon = filtered_data["longitude"].mean()

        # Lag et Folium-kart med gjennomsnittlig senterpunkt
        m = folium.Map(location=[avg_lat, avg_lon], zoom_start=10)

        # Legg til punktene i kartet
        for _, row in filtered_data.iterrows():
            lat, lon = row["latitude"], row["longitude"]
            if not pd.isna(lat) and not pd.isna(lon):  # Sjekk at verdiene ikke er NaN
                ship_name = row.get("ship_name", "Ukjent skip")  # Erstatt med riktig kolonnenavn for skipets navn
                popup_text = f"Timestamp: {row['date_time_utc']}<br>Ship: {ship_name}"

                folium.Marker(
                    location=[lat, lon],
                    popup=popup_text,
                ).add_to(m)

        # Vis kartet
        display(m)
    else:
        print("Ingen 'latitude'/'longitude'-kolonner funnet i datasettet.")

Fant 13 rader i tidsintervallet:


,date_time_utc,mmsi,longitude,latitude,status,course_over_ground,speed_over_ground,rate_of_turn,maneuvre,imo,...,ship_type,length,draught,data_source,ais_class,hex_7,hex_14,geometry,hour,__index_level_0__
0,2024-12-01 11:13:58,258012180,8.034695,58.146992,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664495,"[1, 1, 0, 0, 0, 96, 171, 4, 139, 195, 17, 32, ...",11,3969
1,2024-12-01 11:13:48,258012180,8.034695,58.146987,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664455,"[1, 1, 0, 0, 0, 96, 171, 4, 139, 195, 17, 32, ...",11,3970
2,2024-12-01 11:13:39,258012180,8.034697,58.146982,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664455,"[1, 1, 0, 0, 0, 251, 245, 183, 194, 195, 17, 3...",11,3971
3,2024-12-01 11:13:27,258012180,8.034702,58.146982,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664455,"[1, 1, 0, 0, 0, 66, 162, 125, 106, 196, 17, 32...",11,3972
4,2024-12-01 11:13:19,258012180,8.034703,58.146980,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664455,"[1, 1, 0, 0, 0, 24, 211, 134, 162, 196, 17, 32...",11,3973
5,2024-12-01 11:13:08,258012180,8.034703,58.146978,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664455,"[1, 1, 0, 0, 0, 24, 211, 134, 162, 196, 17, 32...",11,3974
6,2024-12-01 11:12:58,258012180,8.034702,58.146975,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664455,"[1, 1, 0, 0, 0, 66, 162, 125, 106, 196, 17, 32...",11,3975
7,2024-12-01 11:13:50,258258500,8.778725,58.461933,0,231.7,0.0,0.0,0.0,0,...,51,17,1.0,G,A,608155183335079935,639680380713654687,"[1, 1, 0, 0, 0, 187, 39, 15, 11, 181, 142, 33,...",11,15462
8,2024-12-01 11:13:41,258258500,8.778723,58.461930,0,233.9,0.0,0.0,0.0,0,...,51,17,1.0,G,A,608155183335079935,639680380713654679,"[1, 1, 0, 0, 0, 229, 246, 5, 211, 180, 142, 33...",11,15463
9,2024-12-01 11:13:30,258258500,8.778703,58.461888,0,232.4,0.1,0.0,0.0,0,...,51,17,1.0,G,A,608155183335079935,639680380713776863,"[1, 1, 0, 0, 0, 201, 69, 239, 51, 178, 142, 33...",11,15464


# Nedlasting av fil, viser størrlese på filer og hastighet (NBNB endre til at den leser hele mappen?)

In [45]:

def sjekk_filstørrelse(filsti):
    """ Returnerer størrelsen på en fil i MB eller KB """
    if not os.path.exists(filsti):
        return "Fil ikke funnet"

    filstørrelse_bytes = os.path.getsize(filsti)
    if filstørrelse_bytes > 100_000:
        return f"{filstørrelse_bytes / (1024 * 1024):.2f} MB"
    else:
        return f"{filstørrelse_bytes / 1024:.2f} KB"

def sammenlign_filstørrelser(original_fil, partisjonert_fil, filtrert_fil):
    """ Sammenligner filstørrelsene før og etter filtrering/partisjonering """
    print(f"Original filstørrelse: {sjekk_filstørrelse(original_fil)}")
    print(f"Partisjonert filstørrelse: {sjekk_filstørrelse(partisjonert_fil)}")
    print(f"Filstørrelse etter filtrering: {sjekk_filstørrelse(filtrert_fil)}")

# Definer filstier
original_fil = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/hais_2024-12-01.snappy.parquet.geo.parquet"
partisjonert_fil = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataTimer/partitioned_11.parquet"
filtrert_fil = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/FiltrertData/filtered_ship.parquet"

# Opprett DuckDB-tilkobling
con = duckdb.connect()

# Definer tidsintervall og skipsnavn for filtrering
start_time = "2024-12-01 11:49:51"
end_time = "2024-12-01 11:59:00"
ship_name_filter = "RESCUE 157 BILL"

### Mål tid for å hente data fra originalfilen ###
start_original = time.time()
query_original = f"""
    SELECT *
    FROM read_parquet('{original_fil}')
    WHERE CAST(date_time_utc AS TIMESTAMP) BETWEEN TIMESTAMP '{start_time}' AND TIMESTAMP '{end_time}'
    AND ship_name = '{ship_name_filter}'
"""
filtered_data_original = con.execute(query_original).fetchdf()
end_original = time.time()
elapsed_original = end_original - start_original
print(f"Tid brukt på å hente data fra ORIGINALFILEN: {elapsed_original:.2f} sekunder")

###Mål tid for å hente data fra partisjonert fil ###
start_partisjonert = time.time()
query_partisjonert = f"""
    SELECT *
    FROM read_parquet('{partisjonert_fil}')
    WHERE CAST(date_time_utc AS TIMESTAMP) BETWEEN TIMESTAMP '{start_time}' AND TIMESTAMP '{end_time}'
    AND ship_name = '{ship_name_filter}'
"""
filtered_data_partisjonert = con.execute(query_partisjonert).fetchdf()
end_partisjonert = time.time()
elapsed_partisjonert = end_partisjonert - start_partisjonert
print(f"Tid brukt på å hente data fra PARTISJONERT FIL: {elapsed_partisjonert:.2f} sekunder")

# Lagre det filtrerte datasettet som en ny Parquet-fil
if not filtered_data_partisjonert.empty:
    filtered_data_partisjonert.to_parquet(filtrert_fil)
    print(f"Filtrert data lagret som: {filtrert_fil}")

# Lukk tilkoblingen
con.close()

# Sammenlign filstørrelser
sammenlign_filstørrelser(original_fil, partisjonert_fil, filtrert_fil)

# Visualisering hvis datasettet har latitude/longitude
if not filtered_data_partisjonert.empty:
    print(f"Fant {len(filtered_data_partisjonert)} rader i tidsintervallet for skipet {ship_name_filter}:")
    display(filtered_data_partisjonert)

    if "latitude" in filtered_data_partisjonert.columns and "longitude" in filtered_data_partisjonert.columns:
        avg_lat = filtered_data_partisjonert["latitude"].mean()
        avg_lon = filtered_data_partisjonert["longitude"].mean()
        m = folium.Map(location=[avg_lat, avg_lon], zoom_start=10)

        for _, row in filtered_data_partisjonert.iterrows():
            lat, lon = row["latitude"], row["longitude"]
            if not pd.isna(lat) and not pd.isna(lon):
                ship_name = row.get("ship_name", "Ukjent skip")
                popup_text = f"Timestamp: {row['date_time_utc']}<br>Ship: {ship_name}"

                folium.Marker(
                    location=[lat, lon],
                    popup=popup_text,
                ).add_to(m)

        display(m)
    else:
        print("Ingen 'latitude'/'longitude'-kolonner funnet i datasettet.")
else:
    print(f"Ingen data funnet for skipet {ship_name_filter} i det spesifiserte tidsintervallet.")


Tid brukt på å hente data fra ORIGINALFILEN: 0.00 sekunder
Tid brukt på å hente data fra PARTISJONERT FIL: 0.00 sekunder
Filtrert data lagret som: /Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/FiltrertData/filtered_ship.parquet
Original filstørrelse: 0.50 MB
Partisjonert filstørrelse: 49.40 KB
Filstørrelse etter filtrering: 17.42 KB
Fant 69 rader i tidsintervallet for skipet RESCUE 157 BILL:


,date_time_utc,mmsi,longitude,latitude,status,course_over_ground,speed_over_ground,rate_of_turn,maneuvre,imo,...,ship_type,length,draught,data_source,ais_class,hex_7,hex_14,geometry,hour,__index_level_0__
0,2024-12-01 11:58:58,257565700,7.444183,58.000800,0,278.5,23.4,-7.0,0.0,0,...,51,14,0.0,G,A,608154275318595583,639679472702610471,"[1, 1, 0, 0, 0, 15, 29, 175, 254, 215, 198, 29...",11,103
1,2024-12-01 11:58:57,257565700,7.444385,58.000787,0,278.5,23.4,-48.0,0.0,0,...,51,14,0.0,G,A,608154275318595583,639679472702608687,"[1, 1, 0, 0, 0, 243, 84, 135, 220, 12, 199, 29...",11,104
2,2024-12-01 11:58:54,257565700,7.444775,58.000740,0,282.1,23.0,-40.0,0.0,0,...,51,14,0.0,G,A,608154275318595583,639679472702589839,"[1, 1, 0, 0, 0, 72, 80, 252, 24, 115, 199, 29,...",11,105
3,2024-12-01 11:58:51,257565700,7.445557,58.000633,0,289.6,23.8,46.0,0.0,0,...,51,14,0.0,G,A,608154275318595583,639679472702602447,"[1, 1, 0, 0, 0, 39, 220, 76, 1, 64, 200, 29, 6...",11,106
4,2024-12-01 11:58:45,257565700,7.446935,58.000420,0,288.1,22.8,-16.0,0.0,0,...,51,14,0.0,G,A,608154275318595583,639679472702686223,"[1, 1, 0, 0, 0, 68, 192, 33, 84, 169, 201, 29,...",11,107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,2024-12-01 11:51:00,257565700,7.489688,58.012880,0,230.2,7.5,10.0,0.0,0,...,51,14,0.0,G,A,608154275251486719,639679472637102735,"[1, 1, 0, 0, 0, 216, 84, 138, 219, 112, 245, 2...",11,167
65,2024-12-01 11:50:38,257565700,7.490077,58.012980,0,315.3,0.7,27.0,0.0,0,...,51,14,0.0,G,A,608154275251486719,639679472637105175,"[1, 1, 0, 0, 0, 128, 238, 236, 167, 214, 245, ...",11,168
66,2024-12-01 11:50:30,257565700,7.490108,58.012973,0,296.0,0.8,-16.0,0.0,0,...,51,14,0.0,G,A,608154275251486719,639679472637105327,"[1, 1, 0, 0, 0, 130, 99, 67, 245, 222, 245, 29...",11,169
67,2024-12-01 11:50:20,257565700,7.490153,58.012953,0,331.6,0.1,-7.0,0.0,0,...,51,14,0.0,G,A,608154275251486719,639679472637105311,"[1, 1, 0, 0, 0, 130, 128, 41, 193, 234, 245, 2...",11,170


## Ny Partisjonering år>måned>dag

In [24]:


def opprett_mappestruktur(data_mappe="./data"):
    """
    Oppretter nødvendig mappestruktur for konvertering.

    Args:
        data_mappe: Sti til hovedmappen for data

    Returns:
        dict: Stier til opprettede mapper
    """
    # Definer mappestruktur
    rå_mappe = os.path.join(data_mappe, "raw")
    prosessert_mappe = os.path.join(data_mappe, "processed")

    # Opprett mapper
    os.makedirs(data_mappe, exist_ok=True)
    os.makedirs(rå_mappe, exist_ok=True)
    os.makedirs(prosessert_mappe, exist_ok=True)

    print(f"Mappestruktur opprettet.")

    # Returner stier for senere bruk
    return {
        "data_mappe": os.path.abspath(data_mappe),
        "rå_mappe": os.path.abspath(rå_mappe),
        "prosessert_mappe": os.path.abspath(prosessert_mappe)
    }

def legg_til_tidspartisjoneringskolonner(df, tid_kolonne='date_time_utc'):
    """
    Legger til kolonner for tidspartisjonering (år, måned, dag, time).
    """
    if tid_kolonne not in df.columns:
        print(f"Advarsel: Tidsstempelkolonne '{tid_kolonne}' finnes ikke")
        return df

    if not pd.api.types.is_datetime64_any_dtype(df[tid_kolonne]):
        print(f"Advarsel: Kolonnen '{tid_kolonne}' er ikke en datetime-kolonne")
        return df

    df_med_tid = df.copy()
    df_med_tid['year'] = df_med_tid[tid_kolonne].dt.year
    df_med_tid['month'] = df_med_tid[tid_kolonne].dt.month
    df_med_tid['day'] = df_med_tid[tid_kolonne].dt.day
    # df_med_tid['hour'] = df_med_tid[tid_kolonne].dt.hour

    return df_med_tid

def opprett_geodataframe(df):
    """
    Oppretter en GeoDataFrame fra en DataFrame ved å finne koordinater eller geometrikolonner.
    """
    from shapely import wkt
    # Sjekk for lat/long kolonner
    if 'longitude' in df.columns and 'latitude' in df.columns:
        try:
            return gpd.GeoDataFrame(
                df,
                geometry=gpd.points_from_xy(df.longitude, df.latitude),
                crs="EPSG:4326"
            )
        except Exception as e:
            print(f"Kunne ikke opprette geometri fra lat/long: {e}")

    # Sjekk for andre geometrikolonner
    geom_kolonner = [col for col in df.columns if any(
        term in col.lower() for term in ['geom', 'coord', 'point', 'polygon', 'linestring', 'wkt']
    )]

    for col in geom_kolonner:
        if df[col].dtype != 'object':
            continue

        try:
            geom = df[col].apply(wkt.loads)
            return gpd.GeoDataFrame(df, geometry=geom, crs="EPSG:4326")
        except Exception:
            continue

    return None

def lagre_partisjonert_geoparquet(gdf, målfilsti, partisjon_kolonner):
    """
    Lagrer en GeoDataFrame som partisjonert GeoParquet.
    """
    from shapely import wkb # NB!! Ikke fjern!

    # Konverterer geometri til WKB for å kunne partisjonere
    df_med_wkb = gdf.copy()
    df_med_wkb['geometry_wkb'] = df_med_wkb['geometry'].apply(lambda geom: wkb.dumps(geom))
    df_for_partisjon = df_med_wkb.drop(columns=['geometry'])

    # Utfør partisjonering
    df_for_partisjon.to_parquet(målfilsti, partition_cols=partisjon_kolonner)

    # Konverter hver partisjonert fil tilbake til GeoParquet
    konverter_partisjonerte_filer_til_geoparquet(målfilsti)
    return målfilsti

def opprett_år_mappe(år, målfilsti):
    """Oppretter en mappe for et spesifikt år."""
    år_mappe = os.path.join(målfilsti, f"year={år}")
    os.makedirs(år_mappe, exist_ok=True)
    return år_mappe

def forbered_wkb_dataframe(år_df):
    """Forbereder en dataframe med WKB-konvertert geometri for partisjonering."""
    # Konverter geometri til WKB for å kunne partisjonere
    år_df_wkb = år_df.copy()
    år_df_wkb['geometry_wkb'] = år_df_wkb['geometry'].apply(lambda geom: wkb.dumps(geom))
    år_df_wkb = år_df_wkb.drop(columns=['geometry'])
    return år_df_wkb

def konverter_fil_til_geoparquet(src_file, dst_file):
    """Konverterer en enkelt fil fra WKB-format til GeoParquet."""
    try:
        part_df = pd.read_parquet(src_file)
        if 'geometry_wkb' not in part_df.columns:
            return False

        part_df['geometry'] = part_df['geometry_wkb'].apply(lambda x: wkb.loads(x))
        part_df = part_df.drop(columns=['geometry_wkb'])
        part_gdf = gpd.GeoDataFrame(part_df, geometry='geometry', crs="EPSG:4326")

        # Sørg for at målmappen eksisterer
        os.makedirs(os.path.dirname(dst_file), exist_ok=True)
        part_gdf.to_parquet(dst_file)
        return True
    except Exception as e:
        print(f"Advarsel: Kunne ikke konvertere {src_file} til GeoParquet: {e}")
        return False

def kopier_og_konverter_filer(temp_år_mappe, år_mappe):
    """Kopierer og konverterer filer fra temp-mappen til målmappen."""
    suksess_count = 0
    feil_count = 0

    # Opprett først alle mappene
    for root, dirs, _ in os.walk(temp_år_mappe):
        for directory in dirs:
            src_dir = os.path.join(root, directory)
            rel_path = os.path.relpath(src_dir, temp_år_mappe)
            dst_dir = os.path.join(år_mappe, rel_path)
            os.makedirs(dst_dir, exist_ok=True)

    # Kopier og konverter filene
    for root, _, files in os.walk(temp_år_mappe):
        for file in files:
            src_file = os.path.join(root, file)
            rel_path = os.path.relpath(src_file, temp_år_mappe)
            dst_file = os.path.join(år_mappe, rel_path)

            if konverter_fil_til_geoparquet(src_file, dst_file):
                suksess_count += 1
            else:
                feil_count += 1

    return suksess_count, feil_count

def lagre_hierarkisk_partisjonert_geoparquet(gdf, målfilsti, partisjon_kolonner):
    """
    Lagrer en GeoDataFrame med hierarkisk partisjonering der år kommer først,
    med saniterte verdier for mappenavn.

    Args:
        gdf: GeoDataFrame som skal lagres
        målfilsti: Base målsti
        partisjon_kolonner: Liste av kolonner for partisjonering etter år

    Returns:
        str: Målfilsti hvis vellykket
    """
    # Legg til sanitiseringsfunksjon
    def sanitize_for_partition(value):
        """Saniterer verdier for bruk i mappenavn"""
        if value is None:
            return "null"

        if isinstance(value, str):
            # Erstatt mellomrom og spesialtegn med underscore
            sanitized = value.replace(" ", "_")
            sanitized = sanitized.replace("/", "_")
            sanitized = sanitized.replace("\\", "_")
            sanitized = sanitized.replace("%", "_")
            sanitized = sanitized.replace(":", "_")
            return sanitized

        return str(value)

    # Sjekk om year er en av kolonnene
    if 'year' not in gdf.columns:
        print("Advarsel: 'year' kolonne finnes ikke for hierarkisk partisjonering")
        return lagre_partisjonert_geoparquet(gdf, målfilsti, partisjon_kolonner)

    # Opprett en temporær mappe for midlertidig lagring
    import tempfile
    temp_dir = tempfile.mkdtemp()

    try:
        # Lag en kopi av dataframen for å sanitere verdier direkte
        gdf_kopi = gdf.copy()

        # Sanitér verdiene direkte i kolonnene (utenom 'year')
        kolonner_å_sanitere = [col for col in partisjon_kolonner if col != 'year']

        for kolonne in kolonner_å_sanitere:
            if kolonne in gdf_kopi.columns:
                # Sanitér verdiene direkte i kolonnen
                gdf_kopi[kolonne] = gdf_kopi[kolonne].apply(sanitize_for_partition)

        # Partisjonér på år
        år_grupper = gdf_kopi.groupby('year')

        total_suksess = 0
        total_feil = 0

        # Filtrer ut year fra partisjoneringskolonnene
        andre_kolonner = [col for col in partisjon_kolonner if col != 'year']

        # Behandle hvert år
        for år, år_df in år_grupper:
            år_mappe = opprett_år_mappe(år, målfilsti)

            # Hvis det er andre partisjoneringskolonner, partisjonér videre
            if andre_kolonner:
                # Forbered dataframe for partisjonering
                år_df_wkb = forbered_wkb_dataframe(år_df)

                # Lag temp-mappe for dette året
                temp_år_mappe = os.path.join(temp_dir, f"year={år}")
                os.makedirs(temp_år_mappe, exist_ok=True)

                # Partisjonér data til temp-mappen med saniterte verdier
                år_df_wkb.to_parquet(temp_år_mappe, partition_cols=andre_kolonner)

                # Kopier og konverter filer til endelig plassering
                suksess, feil = kopier_og_konverter_filer(temp_år_mappe, år_mappe)
                total_suksess += suksess
                total_feil += feil
            else:
                # Hvis ingen andre partisjoneringskolonner, lagre direkte
                år_df.to_parquet(os.path.join(år_mappe, "data.parquet"))
                total_suksess += 1

        if total_feil > 0:
            print(f"Info: {total_suksess} filer konvertert, {total_feil} filer med feil")

        print(f"Partisjonert med saniterte verdier for kolonnene: {kolonner_å_sanitere}")

    except Exception as e:
        print(f"Feil under hierarkisk partisjonering: {e}")
        return False
    finally:
        # Rydd opp temp-mappen
        import shutil
        shutil.rmtree(temp_dir)

    return målfilsti

def konverter_partisjonerte_filer_til_geoparquet(rotmappe):
    """
    Konverterer alle partisjonerte parquet-filer til GeoParquet format.
    """
    from shapely import wkb # NB!! Ikke fjern!
    feil_count = 0

    for root, _, files in os.walk(rotmappe):
        for file in files:
            if not file.endswith('.parquet'):
                continue

            parquet_path = os.path.join(root, file)
            try:
                # Les dataframe
                part_df = pd.read_parquet(parquet_path)

                # Hopp over hvis den ikke har geometry_wkb
                if 'geometry_wkb' not in part_df.columns:
                    continue

                # Konverter WKB tilbake til geometri
                part_df['geometry'] = part_df['geometry_wkb'].apply(lambda x: wkb.loads(x))
                part_df = part_df.drop(columns=['geometry_wkb'])

                # Lag GeoDataFrame
                part_gdf = gpd.GeoDataFrame(part_df, geometry='geometry', crs="EPSG:4326")

                # Skriv GeoParquet-filen
                part_gdf.to_parquet(parquet_path)
            except Exception as e:
                print(f"Feil ved konvertering av {parquet_path}: {e}")
                feil_count += 1

    if feil_count > 0:
        print(f"Advarsel: {feil_count} filer kunne ikke konverteres til GeoParquet")

def konverter_parquet_til_geoparquet(filsti, målfilsti, partisjon_kolonner=None, hierarkisk_år=True):
    """
    Konverterer parquet-fil til GeoParquet-format med partisjonering.

    Args:
        filsti: Sti til parquet-filen
        målfilsti: Sti hvor GeoParquet-filen skal lagres
        partisjon_kolonner: Liste av kolonnenavn som skal brukes for partisjonering
        hierarkisk_år: Hvis True, partisjoneres hierarkisk med år først
    """
    try:
        df = pd.read_parquet(filsti)
    except Exception as e:
        print(f"Kunne ikke lese parquet-fil: {e}")
        return False

    # Håndter tidspartisjonering
    tidspartisjonering = False
    if partisjon_kolonner and 'date_time_utc' in partisjon_kolonner:
        partisjon_kolonner.remove('date_time_utc')
        tidspartisjonering = True

    # Legg til tidspartisjoneringskolonner
    if tidspartisjonering:
        df = legg_til_tidspartisjoneringskolonner(df)
        if hierarkisk_år:
            # For hierarkisk partisjonering, year håndteres separat
            # Legg til andre tidskolonner til partisjoneringskolonnene
            tidskolonner = ['month', 'day']
            partisjon_kolonner = tidskolonner + (partisjon_kolonner or [])
        else:
            # Standard flat partisjonering
            partisjon_kolonner = ['year', 'month', 'day'] + (partisjon_kolonner or [])

    # Opprett GeoDataFrame
    gdf = opprett_geodataframe(df)
    if gdf is None:
        return False

    # Sjekk at alle partisjoneringskolonner finnes
    if partisjon_kolonner and not all(col in gdf.columns for col in partisjon_kolonner):
        print(f"Advarsel: Ikke alle partisjoneringskolonner finnes i datasettet")
        manglende = [col for col in partisjon_kolonner if col not in gdf.columns]
        print(f"Manglende kolonner: {manglende}")
        return False

    # Lagre med eller uten partisjonering
    if partisjon_kolonner:
        if hierarkisk_år and tidspartisjonering:
            return lagre_hierarkisk_partisjonert_geoparquet(gdf, målfilsti, partisjon_kolonner)
        else:
            return lagre_partisjonert_geoparquet(gdf, målfilsti, partisjon_kolonner)
    else:
        gdf.to_parquet(målfilsti)
        return målfilsti

def konverter_alle_parquet_filer(data_mappe, partisjon_kolonner=None, hierarkisk_år=True):
    """
    Konverterer alle parquet-filer i en mappe til GeoParquet.
    """
    rå_mappe = os.path.join(data_mappe, "raw")
    prosessert_mappe = os.path.join(data_mappe, "processed")

    # Opprett mapper
    os.makedirs(rå_mappe, exist_ok=True)
    os.makedirs(prosessert_mappe, exist_ok=True)

    resultater = {
        "konvertert": [],
        "feilet": []
    }

    # Finn alle parquet-filer
    parquet_filer = [f for f in os.listdir(rå_mappe)
                     if f.lower().endswith('.parquet') and os.path.isfile(os.path.join(rå_mappe, f))]

    if not parquet_filer:
        print("Ingen parquet-filer funnet i råmappen")
        return resultater

    # Konverter hver fil
    for filnavn in parquet_filer:
        filsti = os.path.join(rå_mappe, filnavn)
        base_filnavn = os.path.splitext(filnavn)[0]
        målfilsti = os.path.join(prosessert_mappe, f"{base_filnavn}.parquet")

        # Konverter filen
        resultat = konverter_parquet_til_geoparquet(
            filsti,
            målfilsti,
            partisjon_kolonner.copy() if partisjon_kolonner else None,
            hierarkisk_år
        )

        if resultat:
            resultater["konvertert"].append(filsti)
        else:
            resultater["feilet"].append(filsti)
            print(f"Kunne ikke konvertere: {filsti}")

    return resultater

def vis_partisjoneringsstruktur(konvertert_sti):
    """
    Viser partisjoneringsstrukturen for en konvertert fil.
    """

    for root, dirs, files in os.walk(konvertert_sti, topdown=True, followlinks=False):
        nivå = root.replace(konvertert_sti, "").count(os.sep)
        innrykk = "    " * (nivå + 1)

        # Vis mappenavnet
        mappe_navn = os.path.basename(root)
        if mappe_navn:  # Ikke vis for rot-mappen
            print(f"{innrykk}- {mappe_navn}")

        # Vis antall filer i dypeste mapper
        if not dirs and files:
            print(f"{innrykk}  Inneholder {len(files)} filer")

def vis_datasett_info(filsti):
    """
    Viser informasjon om et GeoParquet datasett.
    """
    try:
        gdf = gpd.read_parquet(filsti)
        print(f"  Datasett størrelse: {len(gdf)} rader, {len(gdf.columns)} kolonner")
        return gdf
    except Exception as e:
        print(f"  Kunne ikke lese filen: {e}")
        return None

def start_konvertering(data_mappe="./data", partisjon_kolonner=None, hierarkisk_år=True):
    """
    Start konverteringsprosessen fra Parquet til GeoParquet.

    Args:
        data_mappe: Sti til datamappen
        partisjon_kolonner: Liste av kolonner for partisjonering
        hierarkisk_år: Hvis True, partisjoneres først på år, så andre kolonner
    """
    start_tid = time.time()
    print(f"Starter konvertering av parquet-filer i {data_mappe} ...\n")

    # Opprett raw-mappen
    rå_mappe = os.path.join(data_mappe, "raw")
    os.makedirs(rå_mappe, exist_ok=True)

    # Utfør konverteringen
    resultater = konverter_alle_parquet_filer(data_mappe, partisjon_kolonner, hierarkisk_år)

    print("Konvertering og partisjonering utført:")
    print(f"• Total behandlingstid: {time.time() - start_tid:.2f} sekunder")
    print(f"• Konverterte filer: {len(resultater['konvertert'])}")

    return resultater

In [25]:
resultater = start_konvertering(
    data_mappe="/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/sok-og-redningsoperasjoner",
    partisjon_kolonner=["date_time_utc", "ship_type", "ship_name"]
)


Starter konvertering av parquet-filer i /Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/sok- og redningsoperasjoner ...

Partisjonert med saniterte verdier for kolonnene: ['month', 'day', 'ship_type', 'ship_name']
Partisjonert med saniterte verdier for kolonnene: ['month', 'day', 'ship_type', 'ship_name']
Partisjonert med saniterte verdier for kolonnene: ['month', 'day', 'ship_type', 'ship_name']
Partisjonert med saniterte verdier for kolonnene: ['month', 'day', 'ship_type', 'ship_name']
Partisjonert med saniterte verdier for kolonnene: ['month', 'day', 'ship_type', 'ship_name']
Partisjonert med saniterte verdier for kolonnene: ['month', 'day', 'ship_type', 'ship_name']
Partisjonert med saniterte verdier for kolonnene: ['month', 'day', 'ship_type', 'ship_name']
Partisjonert med saniterte verdier for kolonnene: ['month', 'day', 'ship_type', 'ship_name']
Partisjonert med saniterte verdier for kolonnene: ['month', 'day', 'ship_type', 'ship_name']
Partisjonert med sanite

## Ny filterering


In [27]:
def filtrer_ais_data(
    rot_mappe,
    dato=None,          # datetime objekt for dato - påkrevd
    filtre=None         # Dict med kolonner og verdier å filtrere på
):
    """
    Generell filtreringsfunksjon som kan filtrere på alle kolonner.

    Args:
        rot_mappe: Mappen hvor dataene er lagret
        dato: Dato (datetime) som er påkrevd for å finne riktig datapartisjon
        filtre: Dict der nøklene er kolonnenavn og verdiene er filterverdier
                F.eks. {'ship_name': 'AIDANOVA', 'mmsi': [257376700, 257376701]}
    Returns:
        GeoDataFrame med filtrerte data
    """
    # Sanitiseringsfunksjon for mappenavn
    def sanitize_for_partition(value):
        if value is None:
            return "null"
        if isinstance(value, str):
            sanitized = value.replace(" ", "_")
            sanitized = sanitized.replace("/", "_")
            sanitized = sanitized.replace("\\", "_")
            sanitized = sanitized.replace("%", "_")
            sanitized = sanitized.replace(":", "_")
            return sanitized
        return str(value)

    if not dato:
        print("Dato er påkrevd")
        return None

    if filtre is None:
        filtre = {}

    # Dato-komponenter er alltid påkrevd for navigering
    year, month, day = dato.year, dato.month, dato.day

    # Finn basismappe for dato
    dato_mapper = [m for m in os.listdir(rot_mappe) if os.path.isdir(os.path.join(rot_mappe, m)) and m.startswith(f"hais_{dato.strftime('%Y')}")]

    if not dato_mapper:
        print(f"Ingen data funnet for år {year}")
        return None

    # Finn alle parquet-filer rekursivt som matcher datokriteriene
    data_funnet = []

    for dato_mappe in dato_mapper:
        base_path = os.path.join(rot_mappe, dato_mappe)

        # Traverser mappestrukturen rekursivt
        for root, dirs, files in os.walk(base_path):
            # Sjekk om denne mappen inneholder en dato som matcher vår dato
            if (f"year={year}" in root and
                f"month={month}" in root and
                f"day={day}" in root):

                # Legg til alle parquet-filer i denne stien
                for file in files:
                    if file.endswith('.parquet'):
                        data_funnet.append(os.path.join(root, file))

    if not data_funnet:
        print(f"Ingen parquet-filer funnet for dato {dato.strftime('%Y-%m-%d')}")
        return None

    print(f"Fant {len(data_funnet)} parquet-filer for dato {dato.strftime('%Y-%m-%d')}")

    # Les inn og filtrer dataene
    dfs = []
    for fil_sti in data_funnet:
        try:
            # Ekstraher partisjonsverdier fra stien
            partitions = {}
            parts = fil_sti.split(os.sep)
            for part in parts:
                if '=' in part:
                    key, value = part.split('=', 1)
                    partitions[key] = value

            # Les parquet-filen
            df = pd.read_parquet(fil_sti)

            # Legg til partisjonsverdier som kolonner hvis de ikke finnes
            for key, value in partitions.items():
                df[key] = value

            # Filtrer basert på brukerens kriterier
            skip_df = False

            for kolonne, verdi in filtre.items():
                if kolonne not in df.columns:
                    skip_df = True
                    break

                # Sjekk om kolonnen er en partisjonert kolonne
                if kolonne in partitions:
                    # For partisjonerte kolonner må vi sammenligne med den saniterte verdien
                    if isinstance(verdi, list):
                        # Hvis vi filtrerer på flere verdier
                        sanitized_values = [sanitize_for_partition(v) for v in verdi]
                        if partitions[kolonne] not in sanitized_values:
                            skip_df = True
                            break
                    else:
                        # Hvis vi filtrerer på én verdi
                        sanitized_value = sanitize_for_partition(verdi)
                        if partitions[kolonne] != sanitized_value:
                            skip_df = True
                            break
                else:
                    # For ikke-partisjonerte kolonner, filtrer på vanlig måte
                    if isinstance(verdi, list):
                        if not df[kolonne].isin(verdi).any():
                            skip_df = True
                            break
                    else:
                        if not (df[kolonne] == verdi).any():
                            skip_df = True
                            break

            if not skip_df:
                # Gjør ytterligere filtrering på dataframe-nivå
                for kolonne, verdi in filtre.items():
                    if kolonne in df.columns and kolonne not in partitions:
                        if isinstance(verdi, list):
                            df = df[df[kolonne].isin(verdi)]
                        else:
                            df = df[df[kolonne] == verdi]

                if not df.empty:
                    dfs.append(df)

        except Exception as e:
            print(f"Feil ved lesing av {fil_sti}: {str(e)}")

    if not dfs:
        print("Ingen data gjensto etter filtrering")
        return None

    # Slå sammen alle dataframes
    combined_df = pd.concat(dfs, ignore_index=True)
    print(f"Resultatet har {len(combined_df)} rader fra {len(dfs)} filer")

    return combined_df

## Kjører funksjonen for filtrering "Jeg ønsker å vite topp 10 skip som har blitt registret den 02/01/2024 og er innenfor Agder kyst."

In [ ]:
from datetime import datetime

resultat = filtrer_ais_data(
    rot_mappe="/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/sok-og-redningsoperasjoner/processed",
    dato=datetime(2025, 1, 2),
)

# Filtrer DataFrame for å sikre at kun data fra riktig dato beholdes
valgt_dato = datetime(2025, 1, 2).date()
resultat = resultat[resultat["date_time_utc"].dt.date == valgt_dato]

# Sjekk resultatet
if resultat is not None and not resultat.empty:
    print(f"\nFant {len(resultat)} rader for {valgt_dato}.\nTilgjengelige kolonner:")

    try:
        relevante_kolonner = [kol for kol in ['mmsi', 'ship_name', 'date_time_utc', 'longitude', 'latitude']
                              if kol in resultat.columns]

        if relevante_kolonner:
            print(f"\nFørste 10 rader med utvalgte kolonner:")
            print(resultat[relevante_kolonner].head(10))
        else:
            print("\nFørste 10 rader med alle kolonner:")
            print(resultat.head(10))
    except Exception as e:
        print(f"Feil ved visning av data: {str(e)}")
        print(resultat.head(10))
else:
    print(f"Ingen data funnet for {valgt_dato}.")


## Med duckDB

In [49]:

# Konfigurasjon
rot_mappe = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/sok-og-redningsoperasjoner/processed"
valgt_dato = datetime(2025, 1, 2).date()

# Bruk CAST for dato-sammenligning i DuckDB
query = f"""
SELECT *
FROM read_parquet('{rot_mappe}/**/*.parquet', union_by_name=True)
WHERE CAST(date_time_utc AS DATE) = DATE '{valgt_dato}'
"""

# Hent data via DuckDB
con = duckdb.connect()
resultat = con.execute(query).fetchdf()
con.close()

# Resultatsjekk og visning
if resultat is not None and not resultat.empty:
    print(f"\nFant {len(resultat)} rader for {valgt_dato}.\nTilgjengelige kolonner:")

    try:
        relevante_kolonner = [kol for kol in ['mmsi', 'ship_name', 'date_time_utc', 'longitude', 'latitude']
                              if kol in resultat.columns]

        if relevante_kolonner:
            print(f"\nFørste 10 rader med utvalgte kolonner:")
            print(resultat[relevante_kolonner].head(10))
        else:
            print("\nFørste 10 rader med alle kolonner:")
            print(resultat.head(10))
    except Exception as e:
        print(f"Feil ved visning av data: {str(e)}")
        print(resultat.head(10))
else:
    print(f"Ingen data funnet for {valgt_dato}.")



Fant 22008 rader for 2025-01-02.
Tilgjengelige kolonner:

Første 10 rader med utvalgte kolonner:
        mmsi     ship_name       date_time_utc  longitude  latitude
0  257004200  RC_LILLESAND 2025-01-02 05:36:15   8.385000    58.245
1  257004200  RC_LILLESAND 2025-01-02 01:14:06   8.385000    58.245
2  257004200  RC_LILLESAND 2025-01-02 01:14:06   8.385000    58.245
3  257004200  RC_LILLESAND 2025-01-02 11:47:06   8.385000    58.245
4  257004200  RC_LILLESAND 2025-01-02 12:27:14   8.385000    58.245
5  257004200  RC_LILLESAND 2025-01-02 12:27:14   8.385000    58.245
6  257004200  RC_LILLESAND 2025-01-02 12:24:15   8.385000    58.245
7  257004200  RC_LILLESAND 2025-01-02 12:24:15   8.385000    58.245
8  257004200  RC_LILLESAND 2025-01-02 21:48:14   8.383333    58.245
9  257004200  RC_LILLESAND 2025-01-02 21:48:14   8.383333    58.245


## "Jeg vil sette koordinater inn og dermed visualisere i et kart hvilken ship har vært og posisjoner innenfor 1 km. "

In [124]:


# Parametere
ais_file = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/hais_2024-12-01.snappy.parquet.geo.parquet"
center_lat = 58.142359    # Sentrums-breddegrad (f.eks. Agder-kysten)
center_lon = 8.025218   # Sentrums-lengdegrad
radius_m = 10000         # Radius i meter (100 km)
date_filter = "2024-12-01"  # Dato-filter

# Omregning: 1 grad ≈ 111 km (111000 meter)
radius_deg = radius_m / 111000.0

# Kobler til DuckDB og last spatial-extension
con = duckdb.connect()
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")

# SQL-spørring: Henter for hver unik mmsi både den tidligste og den seneste registreringen innenfor bufferen.
query = f"""
WITH filtered AS (
    SELECT mmsi, date_time_utc, geometry
    FROM read_parquet('{ais_file}')
    WHERE CAST(date_time_utc AS DATE) = '{date_filter}'
      AND ST_Distance(geometry, ST_GeomFromText('POINT({center_lon} {center_lat})')) <= {radius_deg}
),
earliest AS (
    SELECT mmsi, MIN(date_time_utc) AS earliest_time
    FROM filtered
    GROUP BY mmsi
),
latest AS (
    SELECT mmsi, MAX(date_time_utc) AS latest_time
    FROM filtered
    GROUP BY mmsi
)
SELECT
    f.mmsi,
    e.earliest_time,
    l.latest_time,
    (SELECT ST_AsText(f2.geometry)
     FROM filtered f2
     WHERE f2.mmsi = f.mmsi
       AND f2.date_time_utc = e.earliest_time
     LIMIT 1) AS earliest_geom,
    (SELECT ST_AsText(f2.geometry)
     FROM filtered f2
     WHERE f2.mmsi = f.mmsi
       AND f2.date_time_utc = l.latest_time
     LIMIT 1) AS latest_geom
FROM filtered f
JOIN earliest e ON f.mmsi = e.mmsi
JOIN latest l ON f.mmsi = l.mmsi
GROUP BY f.mmsi, e.earliest_time, l.latest_time;
"""

# Henter resultatet som et pandas DataFrame
df = con.execute(query).fetchdf()
print("Antall rader (ett per unik mmsi):", df.shape[0])

# Konverter WKT-strengene til Shapely-objekter med wkt.loads
df['earliest_geom'] = df['earliest_geom'].apply(lambda x: wkt.loads(x) if isinstance(x, str) else x)
df['latest_geom']   = df['latest_geom'].apply(lambda x: wkt.loads(x) if isinstance(x, str) else x)

# Ekstraher lengdegrad og breddegrad for begge registreringene
df['earliest_lon'] = df['earliest_geom'].apply(lambda g: g.x if g else None)
df['earliest_lat'] = df['earliest_geom'].apply(lambda g: g.y if g else None)
df['latest_lon']   = df['latest_geom'].apply(lambda g: g.x if g else None)
df['latest_lat']   = df['latest_geom'].apply(lambda g: g.y if g else None)

# Viser tabellen med relevante kolonner
display(df[['mmsi', 'earliest_time', 'latest_time', 'earliest_lon', 'earliest_lat', 'latest_lon', 'latest_lat']])

# Oppretter et interaktivt Folium-kart med sentrum på (center_lat, center_lon)
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Legger til en sirkel (i meter) som viser bufferområdet
folium.Circle(
    location=[center_lat, center_lon],
    radius=radius_m,
    color='blue',
    fill=True,
    fill_opacity=0.0,
    popup="Bufferområde"
).add_to(m)

# Bruker MarkerCluster for å håndtere overlappende markører
marker_cluster = MarkerCluster().add_to(m)

# Legger til to markører per unik mmsi:
# - En oransje markør for den tidligste registreringen med koordinater
# - En blå markør for den seneste registreringen med koordinater
for idx, row in df.iterrows():
    if row['earliest_lat'] is not None and row['earliest_lon'] is not None:
        popup_text = (
            f"Ship {row['mmsi']}<br>"
            f"Tidligste: {row['earliest_time']}<br>"
            f"Koordinater: ({row['earliest_lat']:.6f}, {row['earliest_lon']:.6f})"
        )
        folium.Marker(
            location=[row['earliest_lat'], row['earliest_lon']],
            popup=popup_text,
            icon=folium.Icon(color='orange', icon='play')
        ).add_to(marker_cluster)
    if row['latest_lat'] is not None and row['latest_lon'] is not None:
        popup_text = (
            f"Ship {row['mmsi']}<br>"
            f"Seneste: {row['latest_time']}<br>"
            f"Koordinater: ({row['latest_lat']:.6f}, {row['latest_lon']:.6f})"
        )
        folium.Marker(
            location=[row['latest_lat'], row['latest_lon']],
            popup=popup_text,
            icon=folium.Icon(color='blue', icon='stop')
        ).add_to(marker_cluster)

# Viser det interaktive kartet i Jupyter Notebook
display(m)


Antall rader (ett per unik mmsi): 1


,mmsi,earliest_time,latest_time,earliest_lon,earliest_lat,latest_lon,latest_lat
0,258012180,2024-12-01 00:00:08,2024-12-01 23:59:59,8.034735,58.146973,8.034743,58.146933
